In [2]:
# import required libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [3]:
# declare a list of random values
daily_sales = [23, 45, 11, -1, -34, 0, 12, 33]

In [4]:
# make a pipeline using tensorflow
dataset = tf.data.Dataset.from_tensor_slices(daily_sales)

In [5]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [6]:
# iterate through dataset and show elements
for element in dataset:
    print(element.numpy())

23
45
11
-1
-34
0
12
33


In [7]:
# if want to show selected no of elements
for i in dataset.take(3):
    print(i.numpy())

23
45
11


In [8]:
# now remove negative values from dataset
dataset = dataset.filter(lambda x: x >= 0)

In [9]:
# iterate again and check
for j in dataset.as_numpy_iterator():
    print(j)

23
45
11
0
12
33


In [10]:
# convert above data to pkr as it is in usd
dataset = dataset.map(lambda y: y*279)

In [11]:
for k in dataset.as_numpy_iterator():
    print(k)

6417
12555
3069
0
3348
9207


In [12]:
# shuffle data elements
dataset = dataset.shuffle(2)

for m in dataset.as_numpy_iterator():
    print(m)

6417
12555
0
3069
3348
9207


In [13]:
# we can create batches 

for j in dataset.batch(3):
    print(j.numpy())

[12555  3069     0]
[3348 6417 9207]


In [14]:
# combine all the above operations in just one line and make a pipeline
dataset_pipe = tf.data.Dataset.from_tensor_slices(daily_sales)
dataset_pipe = dataset_pipe.filter(lambda x: x >= 0).map(lambda y: y*279).shuffle(2).batch(3)

In [15]:
dataset_pipe

<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>

In [16]:
for i in dataset_pipe:
    print(i.numpy())

[12555  3069  6417]
[   0 9207 3348]


# Pipeline To Read Image Files

In [17]:
# read images from the directory
images_dataset = tf.data.Dataset.list_files(r"C:\Users\Arfan Shah\Train\*\*", shuffle = True)

# for loop to iterate over directory and then read images
for i in images_dataset.take(5):
    print(i.numpy())

b'C:\\Users\\Arfan Shah\\Train\\Potato___Early_blight\\b35e4c48-e54e-4e42-a399-b236209314f4___RS_Early.B 8231.JPG'
b'C:\\Users\\Arfan Shah\\Train\\Potato___Early_blight\\df7e4d39-fb74-4e6d-9d32-04d7d2165135___RS_Early.B 6700_flipTB.JPG'
b'C:\\Users\\Arfan Shah\\Train\\Potato___Early_blight\\bd4c7de8-0616-45d4-a060-54b8329a9462___RS_Early.B 6902_180deg.JPG'
b'C:\\Users\\Arfan Shah\\Train\\Potato___Late_blight\\9996efa6-219d-4f07-aa12-dc8ca40609b9___RS_LB 2840.JPG'
b'C:\\Users\\Arfan Shah\\Train\\Potato___Early_blight\\2639ad26-c7e9-4bd6-8000-c002f676a4c6___RS_Early.B 7150_flipTB.JPG'


In [18]:
# declare classes in directory
num_classes = ['Early_blight', 'Healthy', 'Late_blight']

In [19]:
# number of images in directory
image_count = len(images_dataset)
image_count

5702

In [20]:
# training size from the actual dataset
train_size = int(image_count*0.8)
train_size

4561

In [21]:
# training dataset
train_dataset = images_dataset.take(train_size)

# test dataset
test_dataset = images_dataset.skip(train_size)

In [22]:
# legth of training and testing dataset
print(len(train_dataset))
print(len(test_dataset))

4561
1141


In [23]:
# file path
string = 'C:\\Users\\Arfan Shah\\Train\\Potato___Late_blight\\d438f9b3-2952-425e-8c5f-a92a21b35f5f___RS_LB 4470_180deg.JPG'
string.split('\\')[-2]

'Potato___Late_blight'

In [24]:
# find labels in train dataset
for i in train_dataset.take(3):
    print(i.numpy())

b'C:\\Users\\Arfan Shah\\Train\\Potato___Healthy\\bde33d08-678c-4f84-b80e-fcb986a9a446___RS_HL 5415_newPixel25.JPG'
b'C:\\Users\\Arfan Shah\\Train\\Potato___Early_blight\\4be99b40-e269-4c69-8c90-ac75514650af___RS_Early.B 6830.JPG'
b'C:\\Users\\Arfan Shah\\Train\\Potato___Healthy\\d1752ed4-a0a7-4075-917d-205c0e785dcf___RS_HL 1911_flipLR.JPG'


In [25]:
# function to find label path

def get_label(file):
    return tf.strings.split(file, os.path.sep)[-2]

In [26]:
# function to find both x and y parts
def process_dataset(file):
    label = get_label(file)
    image = tf.io.read_file(file)

    # decode jpeg image
    image = tf.image.decode_jpeg(image)

    # resize images
    image = tf.image.resize(image, [256, 256])
    return image, label

In [27]:
# now find label for whole training dataset
train_dataset = train_dataset.map(process_dataset)
for image, label in train_dataset.take(1):
    print("Image: ", image)
    print("Label: ", label)

Image:  tf.Tensor(
[[[  9.   0.   1.]
  [169. 157. 161.]
  [167. 155. 159.]
  ...
  [199. 188. 196.]
  [193. 182. 190.]
  [191. 180. 188.]]

 [[  8.   0.   0.]
  [139. 127. 131.]
  [173. 161. 165.]
  ...
  [192. 181. 189.]
  [186. 175. 183.]
  [185. 174. 182.]]

 [[  8.   0.   0.]
  [167. 155. 159.]
  [192. 180. 184.]
  ...
  [194. 183. 191.]
  [188. 177. 185.]
  [189. 178. 186.]]

 ...

 [[ 18.   8.   7.]
  [138. 128. 127.]
  [153. 141. 141.]
  ...
  [204. 191. 198.]
  [212. 199. 206.]
  [188. 175. 182.]]

 [[  9.   0.   0.]
  [143. 133. 132.]
  [179. 167. 167.]
  ...
  [160. 147. 154.]
  [192. 179. 186.]
  [193. 180. 187.]]

 [[ 10.   0.   0.]
  [134. 124. 123.]
  [152. 140. 140.]
  ...
  [198. 185. 192.]
  [197. 184. 191.]
  [182. 169. 176.]]], shape=(256, 256, 3), dtype=float32)
Label:  tf.Tensor(b'Potato___Healthy', shape=(), dtype=string)


In [28]:
# now scale the image data
def scale(image, label):
    return image/255, label

In [29]:
train_dataset = train_dataset.map(scale)

In [32]:
for image, label in train_dataset.take(1):
    print("Image: ", image.numpy()[0])
    print("Label: ", label.numpy())

Image:  [[0.40784314 0.39215687 0.44705883]
 [0.48235294 0.46666667 0.52156866]
 [0.5882353  0.57254905 0.627451  ]
 [0.5058824  0.49019608 0.54509807]
 [0.4862745  0.47058824 0.5254902 ]
 [0.41960785 0.40392157 0.45882353]
 [0.49803922 0.48235294 0.5372549 ]
 [0.52156866 0.5058824  0.56078434]
 [0.4627451  0.44705883 0.5019608 ]
 [0.47058824 0.45490196 0.50980395]
 [0.47843137 0.4627451  0.5176471 ]
 [0.47843137 0.4627451  0.5176471 ]
 [0.4745098  0.45882353 0.5137255 ]
 [0.46666667 0.4509804  0.5058824 ]
 [0.46666667 0.4509804  0.5058824 ]
 [0.47058824 0.45490196 0.50980395]
 [0.49019608 0.4745098  0.5294118 ]
 [0.47058824 0.45490196 0.50980395]
 [0.53333336 0.5176471  0.57254905]
 [0.40784314 0.39215687 0.44705883]
 [0.47058824 0.45490196 0.50980395]
 [0.4745098  0.45882353 0.5137255 ]
 [0.4392157  0.42352942 0.47843137]
 [0.45882353 0.44313726 0.49803922]
 [0.4627451  0.44705883 0.5019608 ]
 [0.4627451  0.44705883 0.5019608 ]
 [0.47058824 0.45490196 0.50980395]
 [0.4862745  0.47058